In [1]:
from os.path import join as pjoin
import pandas as pd
from tqdm import tqdm

dir_template = "dataset/original/{benchmark}/{model}/{attack}"

In [2]:
path = pjoin(
    dir_template.format(**dict(
        benchmark='imdb',
        model='bert',
        attack='bae'
    )),
    "bert-base-uncased-imdb_bae.csv"
)
path

'dataset/original/imdb/bert/bae/bert-base-uncased-imdb_bae.csv'

In [3]:
df = pd.read_csv(path)
df = df[df.result_type=="Successful"]
df

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,Once again Mr. Costner has dragged out a movie...,Once again Mr. Costner has dragged out a movie...,0.000241,0.871085,0,1,0,203,Successful
1,This is an example of why the [[majority]] of ...,This is an example of why the [[lots]] of acti...,0.000183,0.511798,0,1,0,810,Successful
2,"First of all I hate those [[moronic]] rappers,...","First of all I hate those [[other]] rappers, w...",0.000289,0.742746,0,1,0,383,Successful
3,Not even the Beatles could write songs everyon...,Not even the Beatles could write songs everyon...,0.000303,0.530072,0,1,0,442,Successful
4,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.000311,0.595906,0,1,0,313,Successful
...,...,...,...,...,...,...,...,...,...
9990,Although it strays away from the book a little...,Although it strays away from the book a little...,0.001716,0.672632,1,0,1,437,Successful
9991,I'm a fan of independent film. Dialogue driven...,I'm a fan of independent film. Dialogue driven...,0.000253,0.601020,1,0,1,256,Successful
9993,The movie is made in a style that resembles Lo...,The movie is made in a style that resembles Lo...,0.000199,0.891738,1,0,1,65,Successful
9995,This movie gives you more of an idiea how Aust...,This movie gives you more of an idiea how Aust...,0.000554,0.736054,1,0,1,219,Successful


In [4]:
from transformers import BertModel, BertTokenizer, pipeline
ref = "textattack/bert-base-uncased-imdb"

tokenizer = BertTokenizer.from_pretrained(ref, do_lower_case=False)

classifier = pipeline("sentiment-analysis", model=ref)

/Users/gus/Desktop/envs/nlp_project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
adv = "Once again Mr. Costner has dragged out a movie for far greater than necessary. Aside from the terrific sea rescue sequence, of which there are very few I just did not bother about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this point, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. this magic here, it was all I could do to keep from turning it off an hour in."
ori = "Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in."

In [6]:
classifier(adv), classifier(ori)

([{'label': 'LABEL_1', 'score': 0.8710859417915344}],
 [{'label': 'LABEL_0', 'score': 0.9997591376304626}])

In [7]:
import torch

tok, tok_adv = tokenizer(ori), tokenizer(adv)
t, tadv = torch.tensor(tok['input_ids']), torch.tensor(tok_adv['input_ids'])
t, tadv = t.reshape(1, -1), tadv.reshape(1, -1)
t.shape, tadv.shape

(torch.Size([1, 202]), torch.Size([1, 201]))

In [57]:
model = BertModel.from_pretrained(ref)

emb = model(t).last_hidden_state[:, 0, :]#.squeeze()
emb_adv = model(tadv).last_hidden_state[:, 0, :]#.squeeze()
emb.shape, emb_adv.shape

Some weights of the model checkpoint at textattack/bert-base-uncased-imdb were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(torch.Size([1, 768]), torch.Size([1, 768]))

# Sample 10000 sentences from the imdb dataset

Sample 10000 sentences from the imdb dataset

source : https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [9]:
imdb = pd.read_csv("../raw_data/IMDB Dataset.csv")
imdb = imdb.sample(1_000, random_state=42)
imdb.shape

(1000, 2)

Compute their embedding with the model

In [10]:
imdb_list = list()
for review in tqdm(imdb.review.to_numpy().astype(str)):
    with torch.no_grad():
        review = torch.tensor(tokenizer(review)['input_ids']).reshape(1, -1)[:, :512]
        imdb_list.append(
            model(review).last_hidden_state[:, 0, :].squeeze()
            )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1000/1000 [02:47<00:00,  5.98it/s]


In [11]:
imdb_base = torch.stack(imdb_list, 0).numpy()
imdb_base.shape

(1000, 768)

Perform Kernel PCA

In [47]:
from sklearn.decomposition import KernelPCA as kPCA

transformer = kPCA(n_components=2, kernel='cosine')
imdb_pca = transformer.fit_transform(imdb_base)
imdb_pca.shape

(1000, 2)

Fit multivariate normal distribution

In [69]:
mean = np.mean(imdb_pca, axis=0)
covariance = np.cov(imdb_pca, rowvar=False)

from scipy.stats import multivariate_normal
mvn = multivariate_normal(mean=mean, cov=covariance)

# Compare pdf to adversarial examples

In [79]:
df = df.sample(100, random_state=42)
adv_list = list()
clean_series = df.perturbed_text.str.replace('[', '').str.replace(']', '')
for review in tqdm(clean_series.to_numpy().astype(str)):
    with torch.no_grad():
        review = torch.tensor(tokenizer(review)['input_ids']).reshape(1, -1)[:, :512]
        adv_list.append(
            model(review).last_hidden_state[:, 0, :].squeeze()
            )
normal_list = list()
clean_series = df.original_text.str.replace('[', '').str.replace(']', '')
for review in tqdm(clean_series.to_numpy().astype(str)):
    with torch.no_grad():
        review = torch.tensor(tokenizer(review)['input_ids']).reshape(1, -1)[:, :512]
        normal_list.append(
            model(review).last_hidden_state[:, 0, :].squeeze()
            )

/var/folders/fb/c3y45k5d5gdbmk7pnky9y19c0000gn/T/ipykernel_17911/2567680269.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clean_series = df.perturbed_text.str.replace('[', '').str.replace(']', '')
100%|██████████| 100/100 [00:19<00:00,  5.14it/s]
/var/folders/fb/c3y45k5d5gdbmk7pnky9y19c0000gn/T/ipykernel_17911/2567680269.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clean_series = df.original_text.str.replace('[', '').str.replace(']', '')
100%|██████████| 100/100 [00:20<00:00,  4.90it/s]


In [80]:
adv_base = torch.stack(adv_list, 0).numpy()
normal_base = torch.stack(normal_list, 0).numpy()
adv_base.shape, normal_base.shape

((100, 768), (100, 768))

In [81]:
adv_pca = transformer.transform(adv_base)
normal_pca = transformer.transform(normal_base)
adv_pca.shape, normal_pca.shape

((100, 2), (100, 2))

In [82]:
base_pdf, adv_pdf, normal_pdf = list(), list(), list()

for e in tqdm(imdb_pca):
    base_pdf.append(mvn.pdf(e))
for e in tqdm(adv_pca):
    adv_pdf.append(mvn.pdf(e))
for e in tqdm(normal_pca):
    normal_pdf.append(mvn.pdf(e))

100%|██████████| 100/100 [00:00<00:00, 38223.86it/s]


Mean over the training base (independent, non-adversarial inputs)

In [83]:
np.array(base_pdf).mean()

0.43824962193674216

Mean over the adversarial examples

In [84]:
np.array(adv_pdf).mean()

0.2962777868854116

Mean over the original examples (dependent to previous adversarial ones)

In [85]:
np.array(normal_pdf).mean()

0.4712958580312056